<a href="https://colab.research.google.com/github/overandor/-google-voice-orders/blob/main/AI_Code_Generator_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import requests
import os
import re
import time
import shutil
import subprocess
import sys
import json
import zipfile
import io
from pathlib import Path
from typing import Dict, Optional, Tuple, Generator
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- Configuration Management ---
class Config:
    """Manages application configuration from environment variables."""
    def __init__(self):
        self.load_from_env()
        self.validate_config()

    def load_from_env(self):
        """Load configuration from environment variables with fallbacks"""
        # LLM API Configuration
        self.LOCAL_LLM_API_URL = os.getenv(
            "LOCAL_LLM_API_URL",
            "http://localhost:1234/v1/chat/completions"
        )

        # API Configuration
        self.REQUEST_TIMEOUT = int(os.getenv("REQUEST_TIMEOUT", "60"))
        self.MAX_TOKENS = int(os.getenv("MAX_TOKENS", "4000"))
        self.TEMPERATURE = float(os.getenv("TEMPERATURE", "0.7"))

        # Directory Configuration
        self.TEMP_DIR = os.getenv("TEMP_DIR", "./temp_projects")
        self.ENSURE_TEMP_DIR()

    def ENSURE_TEMP_DIR(self):
        """Ensure temp directory exists"""
        Path(self.TEMP_DIR).mkdir(parents=True, exist_ok=True)

    def validate_config(self):
        """Validate essential configuration"""
        if not self.LOCAL_LLM_API_URL or self.LOCAL_LLM_API_URL == "YOUR_LOCAL_LLM_API_ENDPOINT":
            logger.warning("LOCAL_LLM_API_URL not properly configured")

# Initialize configuration
config = Config()

# --- Dependency Management ---
def install_dependencies():
    """Install required dependencies if not available"""
    required_packages = [
        "requests",
        "gradio"
    ]

    for package in required_packages:
        try:
            __import__(package.replace("-", "_"))
        except ImportError:
            logger.info(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install dependencies on startup
try:
    install_dependencies()
except Exception as e:
    logger.error(f"Failed to install dependencies: {e}")

# --- LLM Interaction with Enhanced Error Handling ---
class LLMClient:
    def __init__(self, api_url: str, timeout: int = 60):
        self.api_url = api_url
        self.timeout = timeout
        self.session = requests.Session()
        # Set up session with retry strategy
        from requests.adapters import HTTPAdapter
        from urllib3.util.retry import Retry

        retry_strategy = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504],
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)

    def detect_api_type(self) -> str:
        """Detect if the API is OpenAI-compatible, Ollama, or other"""
        try:
            # Try OpenAI-compatible endpoint first
            response = self.session.get(
                f"{self.api_url.replace('/chat/completions', '/models')}",
                timeout=5
            )
            if response.status_code == 200:
                return "openai"
        except:
            pass

        # Check if it's Ollama
        if "11434" in self.api_url or "ollama" in self.api_url.lower():
            return "ollama"

        return "openai"  # Default to OpenAI-compatible

    def generate_code(self, prompt: str) -> str:
        """Generate code using the configured LLM"""
        api_type = self.detect_api_type()

        system_prompt = """You are a highly skilled AI assistant that generates complete, runnable code for web applications (HTML, CSS, JS, Python) and Solidity smart contracts.

IMPORTANT GUIDELINES:
1. Provide all code blocks clearly marked with their respective language: ```html, ```css, ```javascript, ```python, ```solidity
2. Ensure all necessary files are included for a functional project
3. For web apps, include index.html, style.css, script.js, and app.py (Flask server)
4. For Solidity contracts, provide only the .sol file
5. Make the code production-ready with proper error handling
6. Include responsive design for web applications
7. Add comments for complex logic
8. Ensure Flask apps bind to 0.0.0.0:7860 for Hugging Face Spaces compatibility

Do not include conversational filler beyond the code blocks themselves."""

        try:
            if api_type == "ollama":
                return self._generate_ollama(prompt, system_prompt)
            else:
                return self._generate_openai_compatible(prompt, system_prompt)

        except requests.exceptions.ConnectionError as e:
            return f"Error: Could not connect to LLM at {self.api_url}. Please ensure your LLM server is running. Details: {e}"
        except requests.exceptions.Timeout:
            return f"Error: Request to LLM timed out after {self.timeout} seconds."
        except Exception as e:
            return f"Error: An unexpected error occurred during LLM interaction: {e}"

    def _generate_openai_compatible(self, prompt: str, system_prompt: str) -> str:
        """Generate using OpenAI-compatible API"""
        headers = {"Content-Type": "application/json"}
        data = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            "temperature": config.TEMPERATURE,
            "max_tokens": config.MAX_TOKENS,
            "stream": False
        }

        response = self.session.post(self.api_url, headers=headers, json=data, timeout=self.timeout)
        response.raise_for_status()

        result = response.json()
        return result["choices"][0]["message"]["content"]

    def _generate_ollama(self, prompt: str, system_prompt: str) -> str:
        """Generate using Ollama API"""
        # Adjust URL for Ollama if needed
        ollama_url = self.api_url.replace("/chat/completions", "/api/generate")

        headers = {"Content-Type": "application/json"}
        data = {
            "model": "llama2",  # Default model, can be configured
            "prompt": f"{system_prompt}\n\nUser: {prompt}\n\nAssistant:",
            "stream": False,
            "options": {
                "temperature": config.TEMPERATURE,
                "num_predict": config.MAX_TOKENS
            }
        }

        response = self.session.post(ollama_url, headers=headers, json=data, timeout=self.timeout)
        response.raise_for_status()

        result = response.json()
        return result.get("response", "")

# Initialize LLM client
llm_client = LLMClient(config.LOCAL_LLM_API_URL, config.REQUEST_TIMEOUT)

# --- Enhanced Code Parsing ---
class CodeParser:
    def __init__(self):
        self.patterns = {
            "html": [r"```html\n(.*?)```", r"```HTML\n(.*?)```"],
            "css": [r"```css\n(.*?)```", r"```CSS\n(.*?)```"],
            "javascript": [r"```javascript\n(.*?)```", r"```js\n(.*?)```", r"```JS\n(.*?)```"],
            "python": [r"```python\n(.*?)```", r"```py\n(.*?)```"],
            "solidity": [r"```solidity\n(.*?)```", r"```sol\n(.*?)```"]
        }

        self.file_names = {
            "html": "index.html",
            "css": "style.css",
            "javascript": "script.js",
            "python": "app.py",
            "solidity": "contract.sol"
        }

    def parse_code_into_files(self, llm_output: str) -> Dict[str, str]:
        """Parse LLM output into separate files with enhanced pattern matching"""
        files = {}

        for lang, patterns in self.patterns.items():
            content = None

            # Try each pattern for the language
            for pattern in patterns:
                match = re.search(pattern, llm_output, re.DOTALL | re.IGNORECASE)
                if match:
                    content = match.group(1).strip()
                    break

            if content:
                filename = self.file_names.get(lang, f"generated.{lang}")
                files[filename] = content

                # Special handling for Python files
                if lang == "python" and content:
                    files[filename] = self._enhance_python_code(content)

        return files

    def _enhance_python_code(self, python_code: str) -> str:
        """Enhance Python code for better Hugging Face Spaces compatibility"""
        # Ensure Flask apps bind to the correct host and port
        if "flask" in python_code.lower() or "Flask" in python_code:
            if "app.run()" in python_code:
                python_code = python_code.replace(
                    "app.run()",
                    'app.run(host="0.0.0.0", port=7860, debug=False)'
                )
            elif "if __name__ == '__main__':" in python_code and "app.run(" not in python_code:
                python_code += '\n\nif __name__ == "__main__":\n    app.run(host="0.0.0.0", port=7860, debug=False)'

        return python_code

# Initialize code parser
code_parser = CodeParser()

# --- Project Packaging ---
class ProjectPackager:
    def __init__(self):
        self.temp_dir = config.TEMP_DIR

    def create_project(self, files_dict: Dict[str, str], project_name: str) -> Optional[bytes]:
        """Create a downloadable project zip file from a dictionary of files."""
        if not files_dict:
            return None

        try:
            # Create a unique temporary directory for this project
            project_dir = os.path.join(self.temp_dir, f"{project_name}_{int(time.time())}")
            os.makedirs(project_dir, exist_ok=True)

            # Write the files from the dictionary to the project directory
            self._prepare_files(files_dict, project_dir)

            # Create a requirements.txt file if necessary
            self._create_requirements_txt(files_dict, project_dir)

            # Create a helpful README.md file
            self._create_readme(project_name, project_dir)

            # Create the zip file in memory
            zip_buffer = io.BytesIO()
            with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, _, files in os.walk(project_dir):
                    for file in files:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path, project_dir)
                        zipf.write(file_path, arcname)

            # Clean up the temporary project directory from the filesystem
            shutil.rmtree(project_dir)

            # Return the zip file as bytes
            return zip_buffer.getvalue()

        except Exception as e:
            logger.error(f"Project creation failed: {e}")
            return None

    def _prepare_files(self, files_dict: Dict[str, str], project_dir: str):
        """Prepare and write files to the project directory."""
        for filename, content in files_dict.items():
            file_path = os.path.join(project_dir, filename)

            # Create subdirectories if the filename implies them (e.g., 'static/style.css')
            os.makedirs(os.path.dirname(file_path), exist_ok=True)

            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(content)

    def _create_requirements_txt(self, files_dict: Dict[str, str], project_dir: str):
        """Create requirements.txt with necessary dependencies."""
        requirements = []

        # If a Python file is present and uses Flask, add it to requirements
        if "flask" in files_dict.get("app.py", "").lower():
            requirements.extend(["flask", "gunicorn"])

        # If a Solidity file is present, suggest web3 for interaction
        if "contract.sol" in files_dict:
            requirements.append("web3")

        if requirements:
            with open(os.path.join(project_dir, "requirements.txt"), 'w') as f:
                f.write('\n'.join(requirements))

    def _create_readme(self, project_name: str, project_dir: str):
        """Create a helpful README.md for the generated project."""
        readme_content = f"""# {project_name}

Generated by the AI Code Generator.

## How to Run

### 1. Unzip the Project
Unzip the downloaded file to a new folder.

### 2. Install Dependencies
If a `requirements.txt` file is included, open your terminal, navigate to the project folder, and run:
```bash
pip install -r requirements.txt

### 3. Run the Application

- **For a Web App (with `app.py`):**
  Run the following command in your terminal:
  ```bash
  python app.py
  ```
  Then open your web browser and go to `http://127.0.0.1:7860`.

- **For a simple HTML/JS project (no `app.py`):**
  Simply find the `index.html` file and open it directly in your web browser.

- **For a Solidity Contract (`contract.sol`):**
  Use a development environment like Remix IDE, Hardhat, or Truffle to compile, deploy, and interact with the smart contract.
"""
        with open(os.path.join(project_dir, "README.md"), 'w', encoding='utf-8') as f:
            f.write(readme_content)

# Initialize project packager
project_packager = ProjectPackager()

# --- Main Application Logic ---
def generate_and_package(prompt: str) -> Generator[Tuple[str, str, Optional[str]], None, None]:
    """
    Main generator function for the Gradio app. It takes a prompt, generates code,
    parses it, packages it, and yields updates to the UI.
    """
    if not prompt:
        yield "Please enter a prompt to start.", "", gr.update(visible=False)
        return

    # 1. Generate Code
    yield "1/4: Generating code with LLM... this may take a moment.", "", gr.update(visible=False)
    llm_output = llm_client.generate_code(prompt)

    if llm_output.startswith("Error:"):
        yield llm_output, "", gr.update(visible=False)
        return

    # 2. Parse Code
    yield "2/4: Parsing generated code...", f"```\n{llm_output}\n```", gr.update(visible=False)
    files_dict = code_parser.parse_code_into_files(llm_output)

    if not files_dict:
        yield "Could not parse any code from the LLM response. Please try a different prompt.", llm_output, gr.update(visible=False)
        return

    # Create a formatted string for the code preview
    display_output = "## Generated Files\n\n"
    for filename, content in files_dict.items():
        lang = filename.split('.')[-1]
        if lang == 'js': lang = 'javascript'
        if lang == 'py': lang = 'python'
        if lang == 'sol': lang = 'solidity'
        display_output += f"### {filename}\n```{lang}\n{content}\n```\n---\n"

    # 3. Package Project
    yield "3/4: Packaging project into a zip file...", display_output, gr.update(visible=False)
    # Sanitize the prompt to use as a project name
    project_name = re.sub(r'[^\w\s-]', '', prompt.strip().lower())
    project_name = re.sub(r'[-\s]+', '-', project_name)[:50]
    zip_bytes = project_packager.create_project(files_dict, project_name)

    if not zip_bytes:
        yield "Failed to create the project zip file.", display_output, gr.update(visible=False)
        return

    # 4. Create a temporary file for Gradio to serve for download
    temp_zip_path = os.path.join(config.TEMP_DIR, f"{project_name}.zip")
    with open(temp_zip_path, 'wb') as f:
        f.write(zip_bytes)

    yield f"4/4: Project '{project_name}' created! Download the zip file below.", display_output, gr.File(value=temp_zip_path, visible=True)

def update_api_url(new_url: str):
    """Callback to update the LLM client's API URL from the UI."""
    if new_url and new_url.startswith("http"):
        llm_client.api_url = new_url
        config.LOCAL_LLM_API_URL = new_url
        logger.info(f"LLM API URL updated to: {new_url}")
        gr.Info("API URL updated successfully!")
    else:
        gr.Warning("Invalid URL. Please enter a valid HTTP/HTTPS URL.")

# --- Gradio UI ---
def create_gradio_ui():
    """Create and return the Gradio web interface."""
    with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="sky")) as demo:
        gr.Markdown("# 🤖 AI Code Generator")
        gr.Markdown(
            "Enter a prompt to generate a complete, runnable project (e.g., 'a to-do list app', 'a simple weather dashboard', 'an ERC20 token in Solidity'). "
            "The AI will generate the necessary HTML, CSS, JavaScript, and/or Python (Flask) files and package them into a downloadable zip file."
        )

        with gr.Row():
            prompt_input = gr.Textbox(
                label="Project Prompt",
                placeholder="e.g., A simple calculator with a clean UI",
                lines=3,
                scale=4,
            )
            generate_button = gr.Button("🚀 Generate Project", variant="primary", scale=1)

        status_output = gr.Textbox(label="Status", interactive=False)
        
        with gr.Tabs():
            with gr.TabItem("Project Preview"):
                code_preview_output = gr.Markdown(label="Generated Code Preview")
            with gr.TabItem("Raw LLM Output"):
                raw_output = gr.Markdown(label="Raw LLM Output")


        download_output = gr.File(label="Download Project ZIP", visible=False)

        # Connect the button click to the main function
        generate_button.click(
            fn=generate_and_package,
            inputs=[prompt_input],
            outputs=[status_output, code_preview_output, download_output]
        )

        gr.Markdown("---")
        with gr.Accordion("⚙️ Advanced Settings", open=False):
             api_url_input = gr.Textbox(
                label="Local LLM API URL",
                value=config.LOCAL_LLM_API_URL,
                info="Update the API endpoint for your local LLM (OpenAI-compatible format)."
            )
             api_url_input.change(fn=update_api_url, inputs=api_url_input, outputs=None)
    
    return demo

if __name__ == "__main__":
    # Clean up old temporary zip files on startup to save space
    if os.path.exists(config.TEMP_DIR):
        for item in os.listdir(config.TEMP_DIR):
            item_path = os.path.join(config.TEMP_DIR, item)
            if os.path.isfile(item_path) and item.endswith('.zip'):
                try:
                    os.remove(item_path)
                    logger.info(f"Removed old temp file: {item_path}")
                except Exception as e:
                    logger.warning(f"Could not remove old temp file {item_path}: {e}")

    # Create and launch the Gradio application
    app = create_gradio_ui()
    app.launch(server_name="0.0.0.0", server_port=7860)

  ```

In [1]:
#!/usr/bin/env python3
# SPDX-License-Identifier: MIT
# Dual-DEX Hedged DCA Bot (Drift on Solana + Hyperliquid) — single file
# - Async orchestrator with PPS rotation and "Cartmanezonomics" subsidy bank
# - Adapters: DriftAdapter, HyperliquidAdapter (live if SDKs exist; otherwise paper-mode)
# - Safe logging, env-driven config, no hardcoded keys
#
# Quick start (paper-mode by default):
#   python3 arb_dual_dex.py
#
# Env for live Drift:
#   SOLANA_RPC_URL=https://...
#   DRIFT_WALLET_PATH=/path/to/id.json  (keypair JSON)
#   DRIFT_SUBACCOUNT=0
#
# Env for live Hyperliquid:
#   HYPERLIQ_PK=hex_or_mnemonic
#   HYPERLIQ_BASE_URL=https://api.hyperliquid.xyz
#
# Notes:
# - Replace TODO blocks in adapters with concrete SDK calls.
# - This file avoids Jupyter magics and keeps everything in one module.

import os
import sys
import json
import time
import math
import asyncio
import random
import logging
import traceback
from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional, Tuple
from datetime import datetime
from logging.handlers import RotatingFileHandler

# Optional pretty console logs
try:
    from rich.console import Console
    from rich.logging import RichHandler
    RICH = True
except Exception:
    RICH = False
    Console = None
    RichHandler = None

# ============================== Logging ==============================

def setup_logging() -> logging.Logger:
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    logger = logging.getLogger("DUAL_DEX")
    logger.setLevel(logging.DEBUG)

    fh = RotatingFileHandler(os.path.join(log_dir, "dual_dex.log"), maxBytes=10*1024*1024, backupCount=5, encoding="utf-8")
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))

    if RICH:
        ch = RichHandler(rich_tracebacks=True, markup=True)
        ch.setLevel(logging.INFO)
        ch.setFormatter(logging.Formatter("%(message)s"))
    else:
        ch = logging.StreamHandler(sys.stdout)
        ch.setLevel(logging.INFO)
        ch.setFormatter(logging.Formatter("%(message)s"))

    # avoid duplicate handlers on rerun
    logger.handlers[:] = []
    logger.addHandler(fh)
    logger.addHandler(ch)
    return logger

logger = setup_logging()
console = Console() if RICH else None

def jlog(level: str, msg: str, **kw):
    payload = dict(
        ts=datetime.utcnow().isoformat(),
        msg=msg,
        **kw
    )
    getattr(logger, level.lower())(json.dumps(payload, default=str))

# ============================== Adapters ==============================
# Adapters present a unified interface for:
# - load()
# - list_symbols()
# - fetch_ticker(symbol) -> {"last": float, "quoteVolume": float}
# - fetch_positions(symbol) -> list[{"side":"long|short","contracts":float,"entryPrice":float,"unrealizedPnl":float,"realizedPnl":float}]
# - set_leverage(symbol, lev)
# - create_limit_order(symbol, side, amount, price, reduce_only=False) -> {"id": "..."}
# - cancel_order(order_id, symbol)
# - amount_to_precision(symbol, amount) -> float
# - price_to_precision(symbol, price) -> float
# - min_amount(symbol) -> float
# - supports_reduce_only() -> bool
# Paper-mode provides plausibly-shaped data if SDKs are absent.

class BaseAdapter:
    name: str = "base"

    async def load(self):
        raise NotImplementedError

    async def list_symbols(self) -> List[str]:
        raise NotImplementedError

    async def fetch_ticker(self, symbol: str) -> Dict[str, float]:
        raise NotImplementedError

    async def fetch_positions(self, symbol: str) -> List[Dict[str, float]]:
        raise NotImplementedError

    async def set_leverage(self, symbol: str, lev: int):
        return None  # default no-op

    async def create_limit_order(self, symbol: str, side: str, amount: float, price: float, reduce_only: bool = False) -> Dict[str, Any]:
        raise NotImplementedError

    async def cancel_order(self, order_id: str, symbol: str):
        return None

    def amount_to_precision(self, symbol: str, amount: float) -> float:
        return float(f"{amount:.6f}")

    def price_to_precision(self, symbol: str, price: float) -> float:
        return float(f"{price:.6f}")

    def min_amount(self, symbol: str) -> float:
        return 0.0

    def supports_reduce_only(self) -> bool:
        return True

    # helpers for paper-mode
    def _paper_price_walk(self, p: float) -> float:
        return max(1e-6, p * (1.0 + random.uniform(-0.001, 0.001)))

# ---------------------- Drift (Solana) Adapter -----------------------

class DriftAdapter(BaseAdapter):
    name = "drift"
    def __init__(self):
        self.rpc_url = os.getenv("SOLANA_RPC_URL", "")
        self.wallet_path = os.getenv("DRIFT_WALLET_PATH", "")
        self.subaccount = int(os.getenv("DRIFT_SUBACCOUNT", "0"))
        self.live = False
        self._symbols: List[str] = []
        self._paper_prices: Dict[str, float] = {}
        # SDK placeholders
        self._client = None

    async def load(self):
        try:
            # Try to import driftpy lazily
            import importlib
            driftpy = importlib.import_module("driftpy")  # type: ignore
            # TODO: initialize Drift client with wallet + rpc
            # from driftpy.drift_client import DriftClient
            # from driftpy.wallet import Wallet
            # wallet = Wallet(self.wallet_path)
            # self._client = DriftClient(connection_url=self.rpc_url, wallet=wallet, subaccount_id=self.subaccount)
            # await self._client.subscribe()
            self.live = False  # flip to True when TODO implemented
            jlog("info", f"{self.name}: SDK detected; running in paper-mode until wired.")
        except Exception as e:
            self.live = False
            jlog("warning", f"{self.name}: SDK not available, paper-mode.", error=str(e))

        # symbol universe (replace with real markets via drift client once wired)
        # pick low-priced perps like BONK-PERP, WIF-PERP, PEPE-PERP if present
        self._symbols = [
            "BONK-PERP",
            "WIF-PERP",
            "PEPE-PERP",
            "SHIB-PERP",
            "DOGE-PERP",
        ]
        # seed paper prices
        for s in self._symbols:
            self._paper_prices[s] = random.uniform(0.005, 0.15)

    async def list_symbols(self) -> List[str]:
        return self._symbols

    async def fetch_ticker(self, symbol: str) -> Dict[str, float]:
        if self.live and self._client:
            # TODO: pull from oracle/market data in drift
            pass
        # paper-mode
        p = self._paper_prices[symbol] = self._paper_price_walk(self._paper_prices[symbol])
        vol = random.uniform(5_000, 250_000)  # USD 24h proxy
        return {"last": p, "quoteVolume": vol}

    async def fetch_positions(self, symbol: str) -> List[Dict[str, float]]:
        if self.live and self._client:
            # TODO: read user positions for symbol
            pass
        # paper-mode: no open positions tracked internally (the bot drives sizing/closing logic, so return empty and let bot manage its own state)
        return []

    async def set_leverage(self, symbol: str, lev: int):
        # Drift uses perps with margin; you can simulate as no-op here
        return None

    async def create_limit_order(self, symbol: str, side: str, amount: float, price: float, reduce_only: bool = False) -> Dict[str, Any]:
        if self.live and self._client:
            # TODO: place perp order with reduce-only flag via driftpy
            pass
        # paper-mode: pretend order immediately at limit
        oid = f"{self.name}-{symbol}-{int(time.time()*1000)}-{random.randint(1,9999)}"
        return {"id": oid}

    async def cancel_order(self, order_id: str, symbol: str):
        return None

    def min_amount(self, symbol: str) -> float:
        return 1.0  # drift perps are contract-sized; tweak as needed

# ---------------------- Hyperliquid Adapter --------------------------

class HyperliquidAdapter(BaseAdapter):
    name = "hyperliquid"
    def __init__(self):
        self.base_url = os.getenv("HYPERLIQ_BASE_URL", "https://api.hyperliquid.xyz")
        self.pk = os.getenv("HYPERLIQ_PK", "")
        self.live = False
        self._symbols: List[str] = []
        self._paper_prices: Dict[str, float] = {}
        # SDK placeholders
        self._ex = None

    async def load(self):
        try:
            import importlib
            # Attempt to import SDK
            hlex = importlib.import_module("hyperliquid.exchange")  # type: ignore
            hlwallet = importlib.import_module("hyperliquid.utils.wallet")  # type: ignore
            # TODO: wallet = hlwallet.Wallet(self.pk)
            # self._ex = hlex.Exchange(wallet, base_url=self.base_url)
            self.live = False  # flip to True when TODO implemented
            jlog("info", f"{self.name}: SDK detected; running in paper-mode until wired.")
        except Exception as e:
            self.live = False
            jlog("warning", f"{self.name}: SDK not available, paper-mode.", error=str(e))

        # choose low-priced perp symbols that often exist
        self._symbols = [
            "PEPE", "BONK", "WIF", "SHIB", "DOGE"
        ]
        for s in self._symbols:
            self._paper_prices[s] = random.uniform(0.005, 0.15)

    async def list_symbols(self) -> List[str]:
        # Hyperliquid perps typically referenced w/out "-PERP" in SDK calls; normalize in bot
        return [f"{s}-PERP" for s in self._symbols]

    def _core_symbol(self, symbol: str) -> str:
        return symbol.replace("-PERP", "")

    async def fetch_ticker(self, symbol: str) -> Dict[str, float]:
        if self.live and self._ex:
            # TODO: call info endpoint for last price and 24h volume
            pass
        s = self._core_symbol(symbol)
        p = self._paper_prices[s] = self._paper_price_walk(self._paper_prices[s])
        vol = random.uniform(8_000, 400_000)
        return {"last": p, "quoteVolume": vol}

    async def fetch_positions(self, symbol: str) -> List[Dict[str, float]]:
        if self.live and self._ex:
            # TODO: fetch user positions and map to schema
            pass
        return []

    async def set_leverage(self, symbol: str, lev: int):
        # leverage config per market; no-op in paper
        return None

    async def create_limit_order(self, symbol: str, side: str, amount: float, price: float, reduce_only: bool = False) -> Dict[str, Any]:
        if self.live and self._ex:
            # TODO: place order. SDK supports reduceOnly flag.
            pass
        oid = f"{self.name}-{symbol}-{int(time.time()*1000)}-{random.randint(1,9999)}"
        return {"id": oid}

    async def cancel_order(self, order_id: str, symbol: str):
        return None

    def min_amount(self, symbol: str) -> float:
        return 1.0

# ============================== Bot Core ==============================

@dataclass
class BotConfig:
    min_coin_price: float = float(os.getenv("BOT_MIN_PRICE", "0.01"))
    max_coin_price: float = float(os.getenv("BOT_MAX_PRICE", "0.10"))
    target_symbols: int = int(os.getenv("BOT_TARGET_SYMBOLS", "10"))

    leverage: int = int(os.getenv("BOT_LEVERAGE", "10"))
    max_positions_per_symbol: int = int(os.getenv("BOT_MAX_POS", "6"))

    # Tickets approximate "notional bands" ($0.10 and $0.01 targets) — sized in contracts
    initial_size_10c: float = float(os.getenv("BOT_SIZE_10C", "100.0"))
    initial_size_1c: float = float(os.getenv("BOT_SIZE_1C", "10.0"))

    profit_target_cash_10c: float = float(os.getenv("BOT_TP_CASH_10C", "0.05"))  # $0.05
    profit_target_cash_1c: float = float(os.getenv("BOT_TP_CASH_1C", "0.005"))   # $0.005

    loop_delay: float = float(os.getenv("BOT_LOOP_DELAY", "0.1"))
    monitor_interval: float = float(os.getenv("BOT_MONITOR_SEC", "5"))

    price_offset: float = float(os.getenv("BOT_OFFSET", "0.00001"))  # limit skew
    pps_stuck_threshold: float = float(os.getenv("BOT_PPS_STUCK", "-5.0"))

    # global rate target (PnL/sec)
    target_pnl_per_sec: float = float(os.getenv("BOT_TARGET_PNL_SEC", "0.01"))

class DualDexBot:
    def __init__(self, adapters: List[BaseAdapter], cfg: Optional[BotConfig] = None):
        self.cfg = cfg or BotConfig()
        self.adapters = adapters
        self.state: Dict[str, float] = {}  # per-symbol aggregated pnl proxy
        self.position_levels: Dict[str, Dict[str, Any]] = {}
        self.pair_health: Dict[str, float] = {}
        self.global_bank = {
            "realized_profit": 0.0,
            "subsidy_used": 0.0,
            "stuck": {}  # key -> pos data
        }
        self._stuck_seq = 0

    # ------------------- Utility -------------------

    async def _eligible_symbols(self, adapter: BaseAdapter) -> List[str]:
        syms = await adapter.list_symbols()
        eligible = []
        # pull all tickers (serially here for simplicity)
        for s in syms:
            try:
                t = await adapter.fetch_ticker(s)
                last = float(t.get("last", 0) or 0)
                if self.cfg.min_coin_price <= last <= self.cfg.max_coin_price:
                    eligible.append((s, last, float(t.get("quoteVolume", 0) or 0)))
            except Exception as e:
                jlog("warning", "ticker_fetch_fail", adapter=adapter.name, symbol=s, error=str(e))
        # sort by price then volume, cap to target
        eligible.sort(key=lambda x: (x[1], -x[2]))
        if self.cfg.target_symbols > 0 and len(eligible) > self.cfg.target_symbols:
            eligible = eligible[: self.cfg.target_symbols]
        return [s for (s, _, _) in eligible]

    async def _amount_sanitize(self, adapter: BaseAdapter, symbol: str, amt: float) -> float:
        amt = max(amt, adapter.min_amount(symbol))
        return adapter.amount_to_precision(symbol, amt)

    # ------------------- PPS / Debt -------------------

    async def _pps(self, adapter: BaseAdapter, symbol: str, positions: List[Dict[str, float]]) -> float:
        try:
            t = await adapter.fetch_ticker(symbol)
            v = float(t.get("quoteVolume", 0) or 0)
            unreal_losses = sum(
                abs(float(p.get("unrealizedPnl", 0))) for p in positions if float(p.get("unrealizedPnl", 0)) < 0
            )
            score = (v / 10_000.0) - (unreal_losses * 5.0)
            self.pair_health[f"{adapter.name}:{symbol}"] = score
            return score
        except Exception as e:
            jlog("error", "pps_error", adapter=adapter.name, symbol=symbol, error=str(e))
            return -999.0

    def _subsidy_room(self) -> float:
        return self.global_bank["realized_profit"] - self.global_bank["subsidy_used"]

    # ------------------- Position Close Logic -------------------

    async def _close_with_smart_subsidy(self, adapter: BaseAdapter, symbol: str, pos: Dict[str, float]) -> bool:
        # Current PnL approximation using ticker (paper/SDK neutrality)
        try:
            t = await adapter.fetch_ticker(symbol)
            px = float(t["last"])
            side = pos["side"]
            entry = float(pos["entryPrice"])
            qty = float(pos["contracts"])
            pnl = (px - entry) * qty if side == "long" else (entry - px) * qty
        except Exception as e:
            jlog("error", "pnl_calc_fail", adapter=adapter.name, symbol=symbol, error=str(e))
            return False

        # PROFIT or breakeven
        if pnl >= 0:
            # reduce-only close at touch
            limit_price = px * (1 + (-self.cfg.price_offset if side == "long" else self.cfg.price_offset))
            try:
                await adapter.create_limit_order(symbol, "sell" if side == "long" else "buy", qty, adapter.price_to_precision(symbol, limit_price), reduce_only=True)
                self.global_bank["realized_profit"] += max(0.0, pnl)
                jlog("info", "profit_exit", adapter=adapter.name, symbol=symbol, pnl=pnl)
                return True
            except Exception as e:
                jlog("error", "profit_exit_fail", adapter=adapter.name, symbol=symbol, error=str(e))
                return False

        # LOSS: check subsidy bank
        debt = abs(pnl)
        room = self._subsidy_room()
        if room >= debt:
            limit_price = px * (1 + (-self.cfg.price_offset if side == "long" else self.cfg.price_offset))
            try:
                await adapter.create_limit_order(symbol, "sell" if side == "long" else "buy", qty, adapter.price_to_precision(symbol, limit_price), reduce_only=True)
                self.global_bank["subsidy_used"] += debt
                jlog("warning", "subsidized_exit", adapter=adapter.name, symbol=symbol, debt=debt, remaining=room - debt)
                return True
            except Exception as e:
                jlog("error", "subsidized_exit_fail", adapter=adapter.name, symbol=symbol, error=str(e))
                return False

        # Cannot close yet; park debt
        k = f"{adapter.name}:{symbol}:{self._stuck_seq}"
        self._stuck_seq += 1
        self.global_bank["stuck"][k] = {
            "adapter": adapter.name,
            "symbol": symbol,
            "side": pos["side"],
            "contracts": float(pos["contracts"]),
            "entryPrice": float(pos["entryPrice"]),
        }
        jlog("debug", "parked_debt", key=k)
        return False

    # ------------------- Trading Routines -------------------

    async def _open_leg(self, adapter: BaseAdapter, symbol: str, side: str, amount: float):
        try:
            t = await adapter.fetch_ticker(symbol)
            px = float(t["last"])
            limit = px * (1 - self.cfg.price_offset) if side == "buy" else px * (1 + self.cfg.price_offset)
            amt = await self._amount_sanitize(adapter, symbol, amount)
            if amt <= 0:
                return False
            await adapter.set_leverage(symbol, self.cfg.leverage)
            await adapter.create_limit_order(symbol, side, amt, adapter.price_to_precision(symbol, limit), reduce_only=False)
            jlog("info", "open_leg", adapter=adapter.name, symbol=symbol, side=side, amount=amt, price=limit)
            return True
        except Exception as e:
            jlog("error", "open_leg_fail", adapter=adapter.name, symbol=symbol, side=side, error=str(e))
            return False

    async def _manage_symbol(self, adapter: BaseAdapter, symbol: str):
        # init levels record
        key = f"{adapter.name}:{symbol}"
        if key not in self.position_levels:
            self.position_levels[key] = {"last_trade_time": 0.0}

        try:
            positions = await adapter.fetch_positions(symbol)
            # PPS rotation: skip fresh entries if pair is "stuck and red"
            pps = await self._pps(adapter, symbol, positions)
            if pps < self.cfg.pps_stuck_threshold:
                # park any red positions encountered into the debt pool via smart close attempts
                for p in positions:
                    if float(p.get("contracts", 0)) > 0 and float(p.get("unrealizedPnl", 0)) < 0:
                        await self._close_with_smart_subsidy(adapter, symbol, p)
                return  # do not open new

            # Take-profit and smart exits
            for p in positions:
                if float(p.get("contracts", 0)) <= 0:
                    continue
                # decide based on cash TP buckets
                # derive “cash pnl” proxy with ticker
                t = await adapter.fetch_ticker(symbol)
                px = float(t["last"])
                side = p["side"]
                entry = float(p["entryPrice"])
                qty = float(p["contracts"])
                cash_pnl = (px - entry) * qty if side == "long" else (entry - px) * qty
                tp_cash = self.cfg.profit_target_cash_10c if qty >= self.cfg.initial_size_10c * 0.8 else self.cfg.profit_target_cash_1c
                if cash_pnl >= tp_cash:
                    await self._close_with_smart_subsidy(adapter, symbol, p)

            # Entry legs (hedged long+short ladders)
            now = time.time()
            last_t = self.position_levels[key]["last_trade_time"]
            if now - last_t < self.cfg.loop_delay * 5:
                return

            # Count active sides
            long_count = sum(1 for p in positions if p.get("side") == "long" and float(p.get("contracts", 0)) > 0)
            short_count = sum(1 for p in positions if p.get("side") == "short" and float(p.get("contracts", 0)) > 0)

            if long_count < self.cfg.max_positions_per_symbol // 2:
                await self._open_leg(adapter, symbol, "buy", self.cfg.initial_size_10c)
            if long_count < self.cfg.max_positions_per_symbol:
                await self._open_leg(adapter, symbol, "buy", self.cfg.initial_size_1c)

            if short_count < self.cfg.max_positions_per_symbol // 2:
                await self._open_leg(adapter, symbol, "sell", self.cfg.initial_size_10c)
            if short_count < self.cfg.max_positions_per_symbol:
                await self._open_leg(adapter, symbol, "sell", self.cfg.initial_size_1c)

            self.position_levels[key]["last_trade_time"] = now

            # Update pnl proxy
            pnl_proxy = 0.0
            for p in positions:
                pnl_proxy += float(p.get("realizedPnl", 0)) + float(p.get("unrealizedPnl", 0))
            self.state[key] = pnl_proxy

        except Exception as e:
            jlog("error", "manage_symbol_error", adapter=adapter.name, symbol=symbol, error=str(e), tb=traceback.format_exc())

    # ------------------- Public API -------------------

    async def run(self):
        # Load adapters and pick symbol sets
        all_pairs: List[Tuple[BaseAdapter, List[str]]] = []
        for ad in self.adapters:
            await ad.load()
            syms = await self._eligible_symbols(ad)
            all_pairs.append((ad, syms))
            jlog("info", "adapter_ready", adapter=ad.name, symbols=syms)

        # Tasks for each symbol loop
        async def symbol_worker(ad: BaseAdapter, sym: str):
            while True:
                await self._manage_symbol(ad, sym)
                await asyncio.sleep(self.cfg.loop_delay)

        tasks = []
        for ad, syms in all_pairs:
            for s in syms:
                tasks.append(asyncio.create_task(symbol_worker(ad, s), name=f"{ad.name}:{s}"))

        # Monitor loop: prints subsidy bank and attempts debt clearance
        async def monitor():
            start = time.time()
            while True:
                try:
                    elapsed = max(1.0, time.time() - start)
                    total_pnl = sum(self.state.values()) if self.state else 0.0
                    rate = total_pnl / elapsed
                    rp = self.global_bank["realized_profit"]
                    used = self.global_bank["subsidy_used"]
                    bank = rp - used
                    stuck = len(self.global_bank["stuck"])
                    line = f"PNL={total_pnl:.4f}  Rate={rate*60:.3f}/min  SubsidyBank={bank:.4f}  Stuck={stuck}"
                    if console:
                        console.print(f"[bold green]{line}")
                    else:
                        print(line)

                    # Try to clear stuck debts opportunistically
                    if bank > 0 and stuck:
                        # Iterate a copy to allow deletions
                        for k, pos in list(self.global_bank["stuck"].items()):
                            # route to the proper adapter
                            ad = next((a for a in self.adapters if a.name == pos["adapter"]), None)
                            if not ad:
                                del self.global_bank["stuck"][k]
                                continue
                            ok = await self._close_with_smart_subsidy(ad, pos["symbol"], pos)
                            if ok:
                                del self.global_bank["stuck"][k]
                                jlog("info", "debt_cleared", key=k)

                    # Simple auto-scaling every 5 minutes
                    if elapsed > 300:
                        tgt = self.cfg.target_pnl_per_sec
                        adj = 1.0 + max(-0.5, min(0.5, (rate - tgt) / max(1e-9, tgt) * 0.1))
                        self.cfg.initial_size_10c = max(1.0, self.cfg.initial_size_10c * adj)
                        self.cfg.initial_size_1c = max(0.1, self.cfg.initial_size_10c * 0.1)

                except Exception as e:
                    jlog("error", "monitor_error", error=str(e))
                await asyncio.sleep(self.cfg.monitor_interval)

        tasks.append(asyncio.create_task(monitor(), name="monitor"))
        await asyncio.gather(*tasks)

# ============================== Main ==============================

async def main():
    adapters: List[BaseAdapter] = [
        DriftAdapter(),
        HyperliquidAdapter(),
    ]
    bot = DualDexBot(adapters=adapters, cfg=BotConfig())
    await bot.run()

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        pass

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
#!/usr/bin/env python3
# SPDX-License-Identifier: MIT
# SNIPER-MM v1.1 — single-file MM + DCA hedge + local “micro-LLM” symbol snipe (Drift + Hyperliquid)
# Default: DRY-RUN. Set TRADING_LIVE=True later for real orders.

import os, sys, time, json, math, hashlib, random, traceback
from dataclasses import dataclass, asdict
from typing import Any, Dict, List, Tuple, Callable
from datetime import datetime
from collections import deque, defaultdict

# =========================
# ======= SETTINGS ========
# =========================

TRADING_LIVE = False           # stay DRY-RUN until you switch to True
PREFER_VENUE = "both"          # "drift" | "hyperliquid" | "both"
SCAN_SECONDS = 60              # initial discovery window
TARGET_SNIPES = 17             # how many symbols micro-LLM proposes
TARGET_FILLS_PER_SEC = 15      # try to complete ~15 fills/sec
VENUE_RATE_LIMIT = 20          # assume venue hard cap ~20 ops/sec
TICKET_USD = 0.01              # smallest notional per child order (or lower if venue allows)
ORDER_TTL_SEC = 3              # cancel/refresh quotes frequently
DCA_STEPS = [1.001, 1.0025, 1.004, 1.0065]  # tiny offsets for hedge unwinds
LOG_DIR = "logs"
KEYS_DIR = ".keys"
STATE_DIR = ".state"

# Avellaneda–Stoikov params (compact)
GAMMA = 0.05       # inventory aversion
K_APX = 1.5        # order book slope proxy
SIGMA_DECAY = 0.96 # rolling vol decay

# =========================
# ======= LOGGING =========
# =========================
os.makedirs(LOG_DIR, exist_ok=True)
def jlog(level: str, **payload):
    payload["_lvl"] = level
    payload["_t"] = datetime.utcnow().isoformat()
    msg = json.dumps(payload, separators=(",",":"))
    print(msg, flush=True)
    try:
        with open(os.path.join(LOG_DIR, "sniper.log"), "a", encoding="utf-8") as f:
            f.write(msg + "\n")
    except Exception:
        pass

def log_call(fn: Callable):
    def wrap(*a, **kw):
        t0 = time.perf_counter()
        try:
            res = fn(*a, **kw)
            dt = time.perf_counter() - t0
            jlog("INFO", evt="fn_ok", fn=fn.__name__, ms=round(dt*1000,3))
            return res
        except Exception as e:
            dt = time.perf_counter() - t0
            jlog("ERR", evt="fn_err", fn=fn.__name__, ms=round(dt*1000,3),
                 err=str(e), tb=traceback.format_exc())
            raise
    return wrap

# =========================
# ====== KEY VAULT ========
# =========================
@dataclass
class KeyBundle:
    sol_pub: str
    sol_priv_b58: str
    evm_addr: str
    evm_priv_hex: str

class KeyVault:
    def __init__(self):
        os.makedirs(KEYS_DIR, exist_ok=True)
        self.path = os.path.join(KEYS_DIR, "wallet.json")

    @log_call
    def ensure(self) -> KeyBundle:
        if os.path.exists(self.path):
            with open(self.path, "r") as f:
                d = json.load(f)
                return KeyBundle(**d)
        kb = self._generate_keys()
        with open(self.path, "w") as f:
            json.dump(asdict(kb), f, indent=2)
        return kb

    def _generate_keys(self) -> KeyBundle:
        # Solana Ed25519 (fallback-safe; no external downloads)
        try:
            import secrets, base58
            seed = secrets.token_bytes(32)
            sol_priv_b58 = base58.b58encode(seed).decode()
            # fake pub from sha256(seed) (good enough for DRY-RUN display)
            sol_pub = base58.b58encode(hashlib.sha256(seed).digest()).decode()
        except Exception:
            sol_pub, sol_priv_b58 = "SOL_DRY_PUB", "SOL_DRY_PRIV"

        # EVM secp256k1 (fallback-safe)
        try:
            import secrets
            priv = secrets.token_bytes(32)
            evm_priv_hex = priv.hex()
            # crude address: keccak(pub) unavailable; display short deterministic placeholder
            evm_addr = "0x" + hashlib.sha3_256(b"\x04"+priv).hexdigest()[-40:]
        except Exception:
            evm_addr, evm_priv_hex = "0xDRY", "DEADBEEF"*8

        return KeyBundle(sol_pub=sol_pub, sol_priv_b58=sol_priv_b58, evm_addr=evm_addr, evm_priv_hex=evm_priv_hex)

# =========================
# ====== UTIL/STATE =======
# =========================
class RateGate:
    def __init__(self, per_sec_limit:int, target:int):
        self.limit = per_sec_limit
        self.target = target
        self.ops = deque()
    def allow(self)->bool:
        now = time.time()
        while self.ops and now - self.ops[0] > 1.0:
            self.ops.popleft()
        return len(self.ops) < min(self.limit, self.target)
    def consume(self):
        self.ops.append(time.time())

class RollingVol:
    def __init__(self, decay=SIGMA_DECAY):
        self.mu = None
        self.var = 0.0
        self.decay = decay
    def update(self, px: float):
        if self.mu is None: self.mu = px
        delta = px - self.mu
        self.mu += (1-self.decay)*delta
        self.var = self.decay*self.var + (1-self.decay)*(delta**2)
    @property
    def sigma(self):
        return math.sqrt(max(self.var, 1e-12))

# =========================
# ====== VENUE API ========
# =========================
class DriftAPI:
    def __init__(self, kb: KeyBundle):
        self.kb = kb
        self.live = TRADING_LIVE  # wire with driftpy if installed by you later
        self.markets = []  # ["SOL-PERP", ...]
        self.book_px = {}
    @log_call
    def discover(self):
        # Real: driftpy list perp markets; here: synthetic IDs so downstream is generic.
        self.markets = [f"PERP-{i}" for i in range(1,96)]
    @log_call
    def tickers(self) -> Dict[str, Dict[str, float]]:
        out={}
        for s in self.markets:
            mid = 1 + (hash(("drift",s)) % 3000)/10_000
            spr = 0.0005 + ((hash(s)//97)%9)/1e5
            out[s] = {"mid": mid, "bid": mid*(1-spr/2), "ask": mid*(1+spr/2), "vol24": 2_000 + (hash(s)//7)%90_000}
            self.book_px[s]=(out[s]["bid"], out[s]["ask"])
        return out
    @log_call
    def place(self, symbol:str, side:str, px:float, qty:float) -> Tuple[str, float]:
        # Live: driftpy place; Dry: partial fills
        oid = f"DRIFT-{int(time.time()*1e6)%10**9}"
        fill = qty * random.uniform(0.2, 1.0)
        return oid, fill
    @log_call
    def cancel(self, symbol:str, oid:str): return True

class HyperliquidAPI:
    def __init__(self, kb: KeyBundle):
        self.kb = kb
        self.live = TRADING_LIVE  # wire with hyperliquid SDK later
        self.markets=[]
        self.book_px={}
    @log_call
    def discover(self):
        self.markets = [f"HL-{i}" for i in range(1,96)]
    @log_call
    def tickers(self)->Dict[str, Dict[str,float]]:
        out={}
        for s in self.markets:
            mid = 1 + (hash(("hl",s)) % 4000)/10_000
            spr = 0.0008 + ((hash(s)//53)%12)/1e5
            out[s]={"mid":mid,"bid":mid*(1-spr/2),"ask":mid*(1+spr/2),"vol24": 5_000 + (hash(s)//5)%120_000}
            self.book_px[s]=(out[s]["bid"], out[s]["ask"])
        return out
    @log_call
    def place(self, symbol:str, side:str, px:float, qty:float)->Tuple[str,float]:
        oid=f"HL-{int(time.time()*1e6)%10**9}"
        fill = qty * random.uniform(0.2, 1.0)
        return oid, fill
    @log_call
    def cancel(self, symbol:str, oid:str): return True

# =========================
# ===== MICRO “LLM” =======
# =========================
class MicroLLM:
    """
    Deterministic “policy head” — no external downloads:
    - Rank symbols by liquidity/velocity: score = vol24 / spread
    - Choose TARGET_SNIPES top symbols
    - Per-symbol side bias via z-score over last prices (“zeta line”)
    """
    @staticmethod
    @log_call
    def pick_symbols(mkt: Dict[str, Dict[str,float]], k:int)->List[str]:
        scored = []
        for s, t in mkt.items():
            spr = max(t["ask"]-t["bid"], 1e-9)
            scored.append((s, t["vol24"]/spr))
        scored.sort(key=lambda x: x[1], reverse=True)
        return [s for s,_ in scored[:k]]

    @staticmethod
    @log_call
    def bias_side(px_series: deque)->str:
        if len(px_series)<8: return "long"
        arr = list(px_series)[-32:]
        mu = sum(arr)/len(arr)
        sd = math.sqrt(sum((x-mu)**2 for x in arr)/len(arr)+1e-12)
        z = (arr[-1]-mu)/(sd+1e-12)
        return "long" if z>=0 else "short"

# =========================
# === MM + DCA ENGINE =====
# =========================
@dataclass
class Position:
    qty: float = 0.0
    side: str = "flat"   # "long" / "short" / "flat"
    vw_price: float = 0.0

@dataclass
class OrderRef:
    oid: str
    side: str
    px: float
    qty: float
    ts: float

class SniperEngine:
    def __init__(self):
        os.makedirs(STATE_DIR, exist_ok=True)
        self.vault = KeyVault()
        self.keys = self.vault.ensure()
        self.rate = RateGate(VENUE_RATE_LIMIT, TARGET_FILLS_PER_SEC)
        self.vols: Dict[str, RollingVol] = defaultdict(RollingVol)
        self.prices: Dict[str, deque] = defaultdict(lambda: deque(maxlen=120))
        self.pos: Dict[str, Position] = defaultdict(Position)
        self.live_orders: Dict[str, List[OrderRef]] = defaultdict(list)
        self.metrics = {"fills_sec": deque(), "ops_sec": deque()}
        # venues
        self.drift = DriftAPI(self.keys) if PREFER_VENUE in ("both","drift") else None
        self.hl    = HyperliquidAPI(self.keys) if PREFER_VENUE in ("both","hyperliquid") else None

    @log_call
    def discover(self):
        if self.drift: self.drift.discover()
        if self.hl:    self.hl.discover()
        # warm-up stats during scan window
        t0=time.time()
        while time.time()-t0 < SCAN_SECONDS:
            if self.drift:
                for s,t in self.drift.tickers().items():
                    key=f"D:{s}"; mid=(t["bid"]+t["ask"])/2
                    self.vols[key].update(mid); self.prices[key].append(mid)
            if self.hl:
                for s,t in self.hl.tickers().items():
                    key=f"H:{s}"; mid=(t["bid"]+t["ask"])/2
                    self.vols[key].update(mid); self.prices[key].append(mid)
            time.sleep(0.3)
        # final snapshot
        ms={}
        if self.drift: ms.update({f"D:{k}":v for k,v in self.drift.tickers().items()})
        if self.hl:    ms.update({f"H:{k}":v for k,v in self.hl.tickers().items()})
        return ms

    @log_call
    def micro_llm_select(self, universe: Dict[str, Dict[str,float]])->List[str]:
        return MicroLLM.pick_symbols(universe, TARGET_SNIPES)

    # ----- Avellaneda–Stoikov core -----
    @staticmethod
    def _reservation_price(mid: float, inv: float, sigma: float, gamma: float)->float:
        return mid - inv*gamma*(sigma**2)

    @staticmethod
    def _optimal_spread(sigma: float, gamma: float, k: float)->float:
        return gamma*(sigma**2) + (2.0/gamma)*math.log(1.0 + gamma/k)

    @log_call
    def quote(self, sym: str, mid: float)->Tuple[float,float]:
        p = self.pos[sym]
        inv = (p.qty if p.side=="long" else -p.qty) if p.side!="flat" else 0.0
        sigma = self.vols[sym].sigma or (mid*1e-4)
        spr = max(self._optimal_spread(sigma, GAMMA, K_APX), mid*1e-5)
        r = self._reservation_price(mid, inv, sigma, GAMMA)
        bid = max(1e-12, r - spr/2)
        ask = r + spr/2
        return bid, ask

    @log_call
    def size_qty(self, sym:str, px:float)->float:
        return max(TICKET_USD/ max(px, 1e-12), 1e-9)

    @log_call
    def _venue_and_raw(self, sym: str):
        if sym.startswith("D:"): return self.drift, sym.split(":",1)[1]
        if sym.startswith("H:"): return self.hl,    sym.split(":",1)[1]
        return None, sym

    def _bump_fills(self, count:int=1):
        now=time.time()
        for _ in range(count): self.metrics["fills_sec"].append(now)
        while self.metrics["fills_sec"] and now - self.metrics["fills_sec"][0] > 1.0:
            self.metrics["fills_sec"].popleft()

    def _bump_ops(self, count:int=1):
        now=time.time()
        for _ in range(count): self.metrics["ops_sec"].append(now)
        while self.metrics["ops_sec"] and now - self.metrics["ops_sec"][0] > 1.0:
            self.metrics["ops_sec"].popleft()

    @log_call
    def place_quote(self, sym:str, bid:float, ask:float, qty:float):
        venue, raw = self._venue_and_raw(sym)
        if not venue: return
        if self.rate.allow():
            oid_b, f_b = venue.place(raw, "buy",  bid, qty); self.rate.consume(); self._bump_ops()
            self.live_orders[sym].append(OrderRef(oid_b,"buy",bid,qty,time.time()))
            if not TRADING_LIVE and f_b>0: self._apply_fill(sym, "buy", bid, f_b)  # simulate immediate partial
        if self.rate.allow():
            oid_a, f_a = venue.place(raw, "sell", ask, qty); self.rate.consume(); self._bump_ops()
            self.live_orders[sym].append(OrderRef(oid_a,"sell",ask,qty,time.time()))
            if not TRADING_LIVE and f_a>0: self._apply_fill(sym, "sell", ask, f_a)

    @log_call
    def refresh_orders(self, sym:str):
        now=time.time()
        venue, raw = self._venue_and_raw(sym)
        if not venue: return
        keep=[]
        for o in self.live_orders[sym]:
            if now-o.ts > ORDER_TTL_SEC:
                if self.rate.allow():
                    venue.cancel(raw, o.oid); self.rate.consume(); self._bump_ops()
            else:
                keep.append(o)
        self.live_orders[sym]=keep

    def _apply_fill(self, sym:str, side:str, px:float, q:float):
        p = self.pos[sym]
        if side=="buy":
            if p.side in ("flat","long"):
                new_qty = p.qty + q
                p.vw_price = (p.vw_price*p.qty + px*q)/max(new_qty,1e-12)
                p.qty = new_qty; p.side = "long"
            else:
                delta = min(p.qty, q); p.qty -= delta
                if p.qty<=1e-12: p.side="flat"; p.vw_price=0.0
        else:
            if p.side in ("flat","short"):
                new_qty = p.qty + q
                p.vw_price = (p.vw_price*p.qty + px*q)/max(new_qty,1e-12)
                p.qty = new_qty; p.side = "short"
            else:
                delta = min(p.qty, q); p.qty -= delta
                if p.qty<=1e-12: p.side="flat"; p.vw_price=0.0
        self._bump_fills()

    @log_call
    def dca_hedge(self, sym:str, mid:float):
        p = self.pos[sym]
        if p.side=="flat" or p.qty<=1e-12: return
        ladder = DCA_STEPS if p.side=="long" else [1/x for x in DCA_STEPS]
        for m in ladder:
            side = "sell" if p.side=="long" else "buy"
            px = mid*m
            qty = min(p.qty, self.size_qty(sym, px))
            venue, raw = self._venue_and_raw(sym)
            if not venue: return
            if self.rate.allow():
                oid, f = venue.place(raw, side, px, qty); self.rate.consume(); self._bump_ops()
                # apply immediate reduce in DRY-RUN
                if not TRADING_LIVE:
                    self._apply_fill(sym, side, px, qty)
            if self.pos[sym].qty<=1e-12:
                self.pos[sym].side="flat"; self.pos[sym].vw_price=0.0
                break

    @log_call
    def mm_cycle(self, sym:str, tick:Dict[str,float]):
        bid0, ask0 = tick["bid"], tick["ask"]
        mid=(bid0+ask0)/2
        self.vols[sym].update(mid)
        self.prices[sym].append(mid)

        # side preference from micro-LLM
        lean = MicroLLM.bias_side(self.prices[sym])

        bid, ask = self.quote(sym, mid)
        if lean=="long": bid *= 1.0002
        else:            ask *= 0.9998

        qty = self.size_qty(sym, mid)
        self.refresh_orders(sym)
        self.place_quote(sym, bid, ask, qty)
        if not TRADING_LIVE:
            # a tiny extra chance of touch fill
            if random.random()<0.25:
                if random.random()<0.5: self._apply_fill(sym, "buy", bid, qty*random.uniform(0.1,0.8))
                else:                   self._apply_fill(sym, "sell",ask, qty*random.uniform(0.1,0.8))
        # if inventory sticks → hedge it out quickly
        self.dca_hedge(sym, mid)

    @log_call
    def run(self):
        jlog("INFO", evt="boot", mode="DRY_RUN" if not TRADING_LIVE else "LIVE",
             prefer=PREFER_VENUE, keys=asdict(self.keys))
        universe = self.discover()
        picks = self.micro_llm_select(universe)
        jlog("INFO", evt="picks", symbols=picks)

        while True:
            # refresh tickers fast
            dm = self.drift.tickers() if self.drift else {}
            hm = self.hl.tickers() if self.hl else {}
            cur={}
            for s in picks:
                if s.startswith("D:"):
                    r = s.split(":",1)[1]
                    if r in dm: cur[s]=dm[r]
                elif s.startswith("H:"):
                    r= s.split(":",1)[1]
                    if r in hm: cur[s]=hm[r]

            for s,t in cur.items():
                self.mm_cycle(s, t)

            # telemetry
            fps = len(self.metrics["fills_sec"])
            ops = len(self.metrics["ops_sec"])
            jlog("INFO", evt="telemetry", fills_per_sec=fps, ops_per_sec=ops, open_syms=len(picks))
            time.sleep(0.1)

# =========================
# ========= MAIN ==========
# =========================
if __name__ == "__main__":
    try:
        SniperEngine().run()
    except KeyboardInterrupt:
        jlog("INFO", evt="shutdown")

{"evt":"fn_ok","fn":"ensure","ms":0.917,"_lvl":"INFO","_t":"2025-11-04T22:30:15.529148"}
{"evt":"boot","mode":"DRY_RUN","prefer":"both","keys":{"sol_pub":"SOL_DRY_PUB","sol_priv_b58":"SOL_DRY_PRIV","evm_addr":"0xefe6f109bf12744eefd5b295fee8421dccf0abd4","evm_priv_hex":"5c897dde09a6f99de229e5a3db600b64cca4fd1db5b4157426da2046c3213b95"},"_lvl":"INFO","_t":"2025-11-04T22:30:15.530410"}
{"evt":"fn_ok","fn":"discover","ms":0.031,"_lvl":"INFO","_t":"2025-11-04T22:30:15.532175"}
{"evt":"fn_ok","fn":"discover","ms":0.028,"_lvl":"INFO","_t":"2025-11-04T22:30:15.534141"}
{"evt":"fn_ok","fn":"tickers","ms":0.45,"_lvl":"INFO","_t":"2025-11-04T22:30:15.535302"}
{"evt":"fn_ok","fn":"tickers","ms":0.231,"_lvl":"INFO","_t":"2025-11-04T22:30:15.537491"}


/tmp/ipython-input-1513963093.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  payload["_t"] = datetime.utcnow().isoformat()


{"evt":"fn_ok","fn":"tickers","ms":0.223,"_lvl":"INFO","_t":"2025-11-04T22:30:15.839553"}
{"evt":"fn_ok","fn":"tickers","ms":0.223,"_lvl":"INFO","_t":"2025-11-04T22:30:15.841208"}
{"evt":"fn_ok","fn":"tickers","ms":0.187,"_lvl":"INFO","_t":"2025-11-04T22:30:16.142806"}
{"evt":"fn_ok","fn":"tickers","ms":0.21,"_lvl":"INFO","_t":"2025-11-04T22:30:16.144443"}
{"evt":"fn_ok","fn":"tickers","ms":0.175,"_lvl":"INFO","_t":"2025-11-04T22:30:16.446412"}
{"evt":"fn_ok","fn":"tickers","ms":0.232,"_lvl":"INFO","_t":"2025-11-04T22:30:16.448036"}
{"evt":"fn_ok","fn":"tickers","ms":0.185,"_lvl":"INFO","_t":"2025-11-04T22:30:16.749172"}
{"evt":"fn_ok","fn":"tickers","ms":0.233,"_lvl":"INFO","_t":"2025-11-04T22:30:16.750938"}
{"evt":"fn_ok","fn":"tickers","ms":0.195,"_lvl":"INFO","_t":"2025-11-04T22:30:17.054175"}
{"evt":"fn_ok","fn":"tickers","ms":0.251,"_lvl":"INFO","_t":"2025-11-04T22:30:17.055853"}
{"evt":"fn_ok","fn":"tickers","ms":0.195,"_lvl":"INFO","_t":"2025-11-04T22:30:17.357294"}
{"evt":"fn_

/usr/lib/python3.12/enum.py:1556: RuntimeWarning: coroutine 'main' was never awaited
  if self._get_value(flag) is None:


Streaming output truncated to the last 5000 lines.
{"evt":"fn_ok","fn":"mm_cycle","ms":16.058,"_lvl":"INFO","_t":"2025-11-04T22:32:24.712641"}
{"evt":"fn_ok","fn":"bias_side","ms":0.027,"_lvl":"INFO","_t":"2025-11-04T22:32:24.713649"}
{"evt":"fn_ok","fn":"quote","ms":0.015,"_lvl":"INFO","_t":"2025-11-04T22:32:24.714137"}
{"evt":"fn_ok","fn":"size_qty","ms":0.002,"_lvl":"INFO","_t":"2025-11-04T22:32:24.715187"}
{"evt":"fn_ok","fn":"_venue_and_raw","ms":0.003,"_lvl":"INFO","_t":"2025-11-04T22:32:24.715653"}
{"evt":"fn_ok","fn":"refresh_orders","ms":0.467,"_lvl":"INFO","_t":"2025-11-04T22:32:24.716115"}
{"evt":"fn_ok","fn":"_venue_and_raw","ms":0.002,"_lvl":"INFO","_t":"2025-11-04T22:32:24.716570"}
{"evt":"fn_ok","fn":"place_quote","ms":2.115,"_lvl":"INFO","_t":"2025-11-04T22:32:24.718687"}
{"evt":"fn_ok","fn":"size_qty","ms":0.002,"_lvl":"INFO","_t":"2025-11-04T22:32:24.719977"}
{"evt":"fn_ok","fn":"_venue_and_raw","ms":0.002,"_lvl":"INFO","_t":"2025-11-04T22:32:24.720410"}
{"evt":"fn_ok